# Flatiron School for Data Science - Capstone Project
* Student name: John J. Cho
* Student pace: full time online
* Instructor name: Rafael Carrasco
* Blog post URL: 

## Data Collection and Cleaning
* Despite the obvious inefficiency in manually coding for the scraping of eBay listings using Beautiful Soup rather than using their API, I desired to gain the experience (and pain :-] ) in doing so.

In [4]:
# Importing libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime as dt

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 500)
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
# eBay scraper function
def scrape_ebay(url,pages):
    """
    Function to scrape auction data from eBay using Beautiful Soup.
    The URL link and number of pages (set to show maximum # of listings - 200) must be defined and a dataframe will be returned
    with populated values for Title, Date, Price, Shipping Cost, Unit Condition and Link to the auction URL.
    """
    titles,dates,prices,shipping,condition,subtitles,links,sellers = [], [], [], [], [], [], [], []
    for page in range(1,pages+1):  #loop thru every page of max results (200)
        
        # Get html page data using BeautifulSoup
        req = requests.get(url.format(page))
        soup = BeautifulSoup(req.content, 'html.parser')
        listings = soup.find_all('li', attrs={'class': 's-item'})

        for listing in listings:  #loop thru each listing on page
            for name in listing.find_all('h3', attrs={'class':"s-item__title"}):  #filter for errors if listing is invalid
                if (str(name.find(text=True, recursive=False))!='None') and (listing!=listings[0]):  #listing[0] is page headers
                    
                    # Get auction title
                    name = listing.find('h3', attrs={'class':"s-item__title"})
                    title = str(name.find(text=True, recursive=False))
                    titles.append(title)
                    
                    # Get ending date
                    date = listing.find('span', attrs={'class':"s-item__ended-date"})
                    date = date.find(text=True)
                    dates.append(date)

                    # Get ending price
                    price = listing.find('span', attrs={'class':"s-item__price"})
                    price = price.find(text=True)
                    price = price.replace('$','')
                    price = price.replace(',','')
                    prices.append(price)

                    # Get shipping info
                    ship = listing.find('span', attrs={'class':"s-item__shipping"})
                    if ship==None: ship = listing.find_all('span', attrs={'class':"s-item__shipping"})
                    else: ship = ship.find(text=True)
                    shipping.append(ship)

                    # Get condition info
                    cond = listing.find('div', attrs={'class':"s-item__subtitle"})
                    #cond2 = cond.find(attrs={'class':"SECONDARY_INFO"})
                    cond = cond.find(text=True)
                    condition.append(cond)

                    # Get auction link to scrape more info
                    link = listing.find_all("a", attrs={'class':'s-item__link'})
                    links.append(str(link))
                    
    # Put all info into dataframe
    scraped = pd.DataFrame({"Date":dates,"Title":titles,"Price":prices,"Shipping":shipping,"Condition":condition,
                            "Link":links})
    scraped.Link = scraped.Link.apply(lambda x: x.split('href="')[1].split('"')[0])  #clean up html link
    
    # Add year 2020 in front and convert all date values to datetime object
    scraped.Date = scraped.Date.apply(lambda x: '2020-'+x)
    scraped.Date = pd.to_datetime(scraped.Date)
    scraped = scraped.sort_values(by=['Date'], ascending=False)
    return scraped

In [230]:
# Filtering for Nintendo Switch, Sold Items, US/Canada, max (200) listings per page, Gray color only
pages = 7  #have to manually check then set how many pages to scrape
url = "https://www.ebay.com/sch/i.html?_oaa=1&_dcat=139971&Model=Nintendo%2520Switch&Color=Gray&_fsrp=1&_nkw=nintendo+switch&LH_Complete=1&rt=nc&Region%2520Code=NTSC%252DU%252FC%2520%2528US%252FCanada%2529&LH_Sold=1&_ipg=200&_pgn={}"
df_gray = scrape_ebay(url,pages)

In [ ]:
# Filtering for other colors (Black, Blue, White, Clear, Green, Multicolor, Orange, Pink, Purple, Red, Yellow, Not Specified)
pages = 4
url = "https://www.ebay.com/sch/i.html?_oaa=1&_dcat=139971&Model=Nintendo%2520Switch&Color=Clear%7CGreen%7CMulticolor%7COrange%7CPink%7CPurple%7CRed%7CYellow%7C%21%7CBlack%7CBlue%7CWhite&_fsrp=1&_nkw=nintendo+switch&LH_Complete=1&Region%2520Code=NTSC%252DU%252FC%2520%2528US%252FCanada%2529&LH_Sold=1&_ipg=200&_pgn={}&rt=nc"
df_misc = scrape_ebay(url,pages)

In [ ]:
# Filtering for Switch Lite only
pages = 6
url = "https://www.ebay.com/sch/i.html?_oaa=1&_dcat=139971&Model=Nintendo%2520Switch%2520Lite&_fsrp=1&_nkw=nintendo+switch&LH_Complete=1&Region%2520Code=NTSC%252DU%252FC%2520%2528US%252FCanada%2529&LH_Sold=1&_ipg=200&_pgn={}&rt=nc"
df_lite = scrape_ebay(url,pages)

In [7]:
df

,Date,Title,Price,Shipping,Condition,Link,Location,Seller,Feedback,FBScore
0,2020-05-13 20:54:00,"BRAND NEW - Nintendo Switch 32GB V2 - Gray Joy-Cons Grey - IN HAND, SHIPS NOW",460.00,15.39,Brand New,https://www.ebay.com/itm/BRAND-NEW-Nintendo-Switch-32GB-V2-Gray-Joy-Cons-Grey-IN-HAND-SHIPS-NOW/...,"['Holland', 'Michigan', 'United States']",tho2216,9.0,90.0
1,2020-05-13 20:50:00,"Like A New 2020 Nintendo Switch With 4 Games (Animal Crossing, Zelda and More)",560.00,5.00,Open Box,https://www.ebay.com/itm/Like-A-New-2020-Nintendo-Switch-With-4-Games-Animal-Crossing-Zelda-and-...,"['Suwanee', 'Georgia', 'United States']",leqazson,10.0,100.0
2,2020-05-13 20:42:00,NEW Nintendo Switch 32GB Gray Console with Neon Red and Neon Blue Joy-Con,489.00,0.00,Brand New,https://www.ebay.com/itm/NEW-Nintendo-Switch-32GB-Gray-Console-with-Neon-Red-and-Neon-Blue-Joy-C...,"['Brooklyn', 'New York', 'United States']",giovannecheverri8,43.0,100.0
3,2020-05-13 20:37:00,Nintendo Switch Neon Blue and Neon Red Joyâ€‘Con Console Gamestop Refurbished,449.95,0.00,Refurbished,https://www.ebay.com/itm/Nintendo-Switch-Neon-Blue-and-Neon-Red-Joy-Con-Console-Gamestop-Refurbi...,"['Fremont', 'California', 'United States']",grtsvc,119.0,100.0
4,2020-05-13 20:33:00,Nintendo Switch Neon Red & Neon Blue Console V2 *NEW IN HAND* SHIPS ASAP,460.00,26.70,Brand New,https://www.ebay.com/itm/Nintendo-Switch-Neon-Red-Neon-Blue-Console-V2-NEW-IN-HAND-SHIPS-ASAP/14...,"['West Palm Beach', 'Florida', 'United States']",ppsgh,24.0,100.0
...,...,...,...,...,...,...,...,...,...,...
14231,2020-02-05 20:31:00,Nintendo Switch 32GB Gray Console with Neon Red and Neon Blue Joy-Con,247.50,26.70,Refurbished,https://www.ebay.com/itm/Nintendo-Switch-32GB-Gray-Console-with-Neon-Red-and-Neon-Blue-Joy-Con-/...,"['Canton', 'Ohio', 'United States']",ryanl1313,2.0,50.0
14232,2020-02-05 20:18:00,Nintendo Switch 32GB Gray Console with Neon Red and Neon Blue Joy-Con,215.50,18.50,Brand New,https://www.ebay.com/itm/Nintendo-Switch-32GB-Gray-Console-with-Neon-Red-and-Neon-Blue-Joy-Con-/...,"['Cambridge', 'Ontario', 'Canada']",mohawk-44,267.0,100.0
14233,2020-02-05 19:34:00,Nintendo Switch with 128gb sd card Gray Console with gray Joy-Con.,160.00,15.00,Pre-Owned,https://www.ebay.com/itm/Nintendo-Switch-with-128gb-sd-card-Gray-Console-with-gray-Joy-Con-/2238...,"['Hartsville', 'South Carolina', 'United States']",jorda.mart,632.0,100.0
14234,2020-02-05 18:35:00,Nintendo Switch Hac001 with Grey Joy-Cons 2189,201.00,16.99,Pre-Owned,https://www.ebay.com/itm/Nintendo-Switch-Hac001-with-Grey-Joy-Cons-2189-/372933762550?_trksid=p2...,"['Burnsville', 'Minnesota', 'United States']",epawnamerica,169492.0,99.9


In [228]:
# Get additional details: item location, seller username, feedback count and score using direct page link
# Not defined as a function so variables can persist during troubleshooting and tendency for web pages to change/disappear
i = 0  #index counter
location, feedback, seller, fbscore = [], [], [], []

for url in df.Link.iloc[i:]:

    # Pull additional details from link to auction details
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    listing = soup.find_all('tr', attrs={'class': 'vi-ht20'})
        
    if listing==[]:  #filtering for older listings that need to grab another link to get to correct listing page
        listing = soup.find('div', attrs={'class': 'nodestar-item-card-details__view'})
        if listing==None:  #filtering for power seller with active store
            listing = soup.find_all('tr', attrs={'class': 'vi-ht20'})
        else:
            link = listing.a.get('href')
            df.Link.iloc[i] = link
            req = requests.get(link)
            soup = BeautifulSoup(req.content, 'html.parser')
            listing = soup.find_all('tr', attrs={'class': 'vi-ht20'})
    
    if listing!=[]:  #filter for different page layouts
            
        # Get item location
        temp = listing[3].find_all('div', attrs={'class': 'u-flL'})
        loc = temp[1].find(text=True)
        if loc=='\n':  #slightly diff page layout
            loc = listing[4].find_all('div', attrs={'class': 'u-flL'})[1].get_text()
        location.append(loc)
        # Get feedback count
        try: feed = listing[4].find_all('a')[1].get_text()  #different page layouts
        except: feed = listing[5].find('div', attrs={'class': 'mbg vi-VR-margBtm3'}).find_all('a')[1].get_text()
        feedback.append(feed)
        # Get seller username
        try: name = listing[4].find('div', attrs={'class': 'mbg vi-VR-margBtm3'}).a.get_text()  #different page layouts
        except: name = listing[5].find('div', attrs={'class': 'mbg vi-VR-margBtm3'}).a.get_text()
        seller.append(name)
        # Get feedback score (need to navigate to seller's page first)
        try: link = listing[4].find('div', attrs={'class':'mbg vi-VR-margBtm3'}).a.get('href')  #different page layouts
        except: link = listing[5].find('div', attrs={'class': 'mbg vi-VR-margBtm3'}).a.get('href')
        req = requests.get(link)
        soup = BeautifulSoup(req.content, 'html.parser')
        listing = soup.find('div', attrs={'class':'perctg'})
        if listing.get_text()=='':  #filter for no feedback in last 12 months which does not report %
            score = 100
        else:
            score = listing.get_text().split('%')[0].split('\t')[2]
        fbscore.append(score)
        
    elif soup.find_all(text='We looked everywhere.') != []:  #filter for dead pages (drop row)
        print(i,'Dead Page!')
        df.drop(df.iloc[i:i+1].index, inplace=True)
        i-=1        
    else:
        print(i)

        # Get item location
        listing = soup.find_all('div', attrs={'class': 'iti-eu-bld-gry'})
        if listing==[]:  #filter for pages requiring another link to listing
            listing = soup.find_all('div', attrs={'class': 'nodestar-item-card-details__view'})
            temp = soup.find_all('div', attrs={'class': 'app-status-messages'})[0].get_text()
            if listing==[] and temp=="We're sorry, something went wrong. Please try again.":  #filter for dead listing page
                print(i,'Dead Page!!!')
                df.drop(df.iloc[i:i+1].index, inplace=True)
            link = listing[0].a.get('href')
            req = requests.get(link)
            soup = BeautifulSoup(req.content, 'html.parser')
            listing = soup.find_all('div', attrs={'class': 'iti-eu-bld-gry'})
        loc = listing[0].find().get_text()
        location.append(loc)
        # Get seller username
        listing = soup.find('div', attrs={'class': 'mbg vi-VR-margBtm3'})
        name = listing.a.get_text()
        seller.append(name)
        # Get feedback count
        feed = listing.find_all('a')[1].get_text()
        feedback.append(feed)
        # Get feedback score
        listing = soup.find('div', attrs={'id':'si-fb'})
        if listing!=None:  #filter for pages with score displayed otherwise need to navigate to seller's page
            score = listing.get_text().split('%')[0]
        elif feed=='0':  #filter for new accounts (zero feedback) since page won't list %
            score = 100
        elif listing==None:  #navigate to seller's page
            link = soup.find('div', attrs={'class':'mbg vi-VR-margBtm3'}).a.get('href')
            req = requests.get(link)
            soup = BeautifulSoup(req.content, 'html.parser')
            listing = soup.find('div', attrs={'class':'perctg'})
            if listing.get_text()=='':  #filter for no feedback in last 12 months which does not report %
                score = 100
            else:
                score = listing.get_text().split('%')[0].split('\t')[2]
        else: 
            print(i,'!!!')
            break
        fbscore.append(score)
    i+=1


156
159
160
163
164
168
169
171
172
174
176
177
178
180
183
186
187
188
189
190
193
196
197
199
201
202
203
204
208
209
212
213
215
216
217
218
220
223
224
225
227
230


In [229]:
# Troubleshooting commands
print(len(location),len(seller),len(feedback),len(fbscore), i)
print(location[-1], seller[-1], feedback[-1], fbscore[-1])

232 232 232 232 232
Waipahu, Hawaii, United States  edolfin 935 100


In [232]:
# Add lists to dataframe
df['Location'] = location
df['Seller'] = seller
df['Feedback'] = feedback
df['FBScore'] = fbscore

### Now we'll clean up the values in the `Shipping` column.
> Note that due to the prevalence of [] values (not as a string but an empty set) the range of values had to be explored manually because the .value_counts() function would throw an error.

The range of values included: Free Shipping, +$xx.xx (x=integer), [] (local pickup)

In [115]:
# Function to convert all 'Shipping' values to float
def clean_shipping(df):
    i = 0  #index counter
    for ship in df.Shipping:
        ship = str(ship)  #to avoid errors due to [] values
        if ship[0].lower() == 'f':  #Free shipping value
            df.Shipping.iloc[i] = 0
        elif ship[0]=='+':  #float value starting with +$ (eg. '+$15.00 shipping')
            df.Shipping.iloc[i] = ship[2:].split(' ')[0]
        else:  #all else are local pickup
            df.Shipping.iloc[i] = 0
        i+=1
    df.Shipping = df.Shipping.astype('float64')
    # Also convert Price, Feedback, FBScore columns to float
    df.Price = df.Price.astype('float64')
    df.Feedback = df.Feedback.astype('float64')
    df.FBScore = df.FBScore.astype('float64')
    return df

In [6]:
df_new = clean_shipping(df_new)
df_new.Shipping.value_counts()

0.00     6866
26.70    1173
23.15     913
20.00     406
15.00     375
         ... 
12.87       1
18.18       1
31.61       1
17.02       1
21.65       1
Name: Shipping, Length: 506, dtype: int64

#### For `Condition`, we have the eBay provided values: Brand New / Pre-Owned / Refurbished / Open Box / Parts Only / New (Other). Some users created their own values (seems by mistake intended for other fields) and upon review they can be lumped into New (Other).

In [117]:
# Function to standardize all 'Condition' values
def clean_condition(df):
    cond_list = ['Brand New', 'Pre-Owned', 'Refurbished', 'Open Box', 'Parts Only', 'New (Other)']
    # Filter for rows with values not in list and change to 'New (Other)'
    i = df[df.Condition.isin(cond_list)==False].index
    df.Condition.loc[i] = 'New (Other)'
    return df

In [234]:
df_new = clean_condition(df_new)
df_new.Condition.value_counts()

Pre-Owned      137
Brand New       66
Open Box        18
Parts Only       6
New (Other)      4
Refurbished      1
Name: Condition, dtype: int64

#### To make EDA easier, let's modify the `Location` values into a list (City, State, Country)

In [119]:
# Function to convert Location values into list
def loc_list(df):
    i = 0  #index counter
    for x in df.Location:
        if len(x.split(','))==2:  #filter for missing state/district, fill in 'STATE'
            df['Location'].iloc[i] = [x.split(',')[0], 'STATE', x.split(',')[1].lstrip()]
        else:
            df['Location'].iloc[i] = [x.split(',')[0], x.split(',')[1].lstrip(), x.split(',')[2].lstrip()]
        i+=1
    return df

In [235]:
df_new = loc_list(df_new)
df_new.head(3)

,Date,Title,Price,Shipping,Condition,Link,Location,Seller,Feedback,FBScore
0,2020-02-24 22:20:00,Nintendo Switch HAC-001(-01) 32GB Console BUNDLE MUST LOOK !,710.00,0.0,Pre-Owned,https://www.ebay.com/itm/Nintendo-Switch-HAC-001-01-32GB-Console-BUNDLE-MUST-LOOK/174192016632?h...,"[Oceanside, California, United States]",arvaldezp_0,90.0,96.9
1,2020-02-24 21:11:00,Nintendo Switch Animal Crossing: New Horizon Special Edition Console Preorder,449.99,0.0,Brand New,https://www.ebay.com/itm/Nintendo-Switch-Animal-Crossing-New-Horizon-Special-Edition-Console-Pre...,"[San Francisco, California, United States]",astrosparky,122.0,100.0
2,2020-02-24 21:10:00,Nintendo Switch Console (Joy-Cons) + Carry Case,249.99,0.0,Pre-Owned,https://www.ebay.com/itm/Nintendo-Switch-Console-Joy-Cons-Carry-Case-/333526212695?_trksid=p2349...,"[Lincoln Park, New Jersey, United States]",twobrothersgames,27761.0,100.0


In [ ]:
# Fix missing location values ([], \n) if needed
for i in df[df.Location=='\n'].index:
    req = requests.get(df.Link.iloc[i])
    soup = BeautifulSoup(req.content, 'html.parser')
    listing = soup.find_all('tr', attrs={'class': 'vi-ht20'})
    df.Location.iloc[i] = listing[4].find_all('div', attrs={'class': 'u-flL'})[1].get_text()

In [245]:
# Fix location values incorrectly stored as string instead of list
notlist = []
i = 0
for x in df.Location:
    if not isinstance(x, list): notlist.append(i)
    i+=1

for i in notlist:
    df.Location.iloc[i] = [df.Location.iloc[i][1:-1].split(',')[0][1:-1],df.Location.iloc[i][1:-1].split(',')[1][2:-1],
                               df.Location.iloc[i][1:-1].split(',')[2][2:-1]]

In [ ]:
df = clean_shipping(df)
df = clean_condition(df)
df = loc_list(df)

In [242]:
# Combining all dataframes
df = pd.concat([df_new, df_gray, df_misc, df])
df.Date = pd.to_datetime(df.Date)
df = df.sort_values('Date',ascending=False).reset_index().drop(columns='index')
df = df.drop_duplicates(['Date','Title','Price','Seller','City'])

### Let's move the Switch Lites incorrectly categorized as 'main' Switch systems to our 'lite' dataframe.

In [274]:
lite = []
i = 0
for t in df.Title:  #record all indexes with 'lite' in title
    if 'lite' in t.lower(): lite.append(i)
    i+=1
print(len(lite))

temp = df.iloc[lite]
df_lite = pd.concat([df_lite, temp])  #add identified listings to lite df
df_lite.Date = pd.to_datetime(df_lite.Date)
df_lite = df_lite.sort_values('Date',ascending=False).reset_index(drop=True)
df_lite = df_lite.drop_duplicates(['Date','Title','Price','Shipping','Seller','Link'])  #drop duplicates
df.drop(temp.index,inplace=True)  #drop identified listings from main df

540

In [328]:
# Pickle throwing errors - convert to csv file to save data
df.to_csv('df.csv',index=False)
df_lite.to_csv('df_lite.csv',index=False)

### Now that we have our data collected and organized, let's start diving into some analysis. Please open the next Jupyter notebook: EDA Model.ipynb